In [5]:
import matplotlib.pyplot as plt
import numpy as np
import math 
import functools
import matplotlib.animation as animation
import matplotlib.text as matText

from matplotlib.collections import PatchCollection
from matplotlib.patches import Arc as patchArc
from matplotlib.ticker import FixedLocator as FixedLocator
from sympy import sieve

%matplotlib auto
#plt.rcParams["animation.html"] = "jshtml"
FRAMES_PER_NUMBER = 20
MAX_NUMBER_TO_CHECK = 50
plotCenter = (0,0)
artists_to_draw_manually = []


fig = plt.figure()
ax = fig.subplots()

numberDisplay = fig.text(.5, .9,'2', fontsize='large', animated=True)
artists_to_draw_manually.append(numberDisplay)

xSpine = ax.spines['bottom']
xSpine.set_position('zero')
ySpine = ax.spines['left']
ySpine.set_position('zero')
ax.spines[['right', 'top']].set_visible(False)

#xSpine.set_bounds(0, None)
backGroundImage = None

class PrimeFixedLocator(FixedLocator):
	primeTicks = []
	oldVmax = 0

	def __init__(self):
		self.locs = []

	def __call__(self):
		return self.tick_values()

	def tick_values(self):
		
		"""
		Return the locations of the ticks.

		.. note::

			Because the values are fixed, vmin and vmax are not used in this
			method.

		"""
		vmin, vmax = self.axis.get_view_interval()

		if (vmax > self.oldVmax) :
			self.primeTicks = list(sieve.primerange(1,vmax))
			self.oldVmax = vmax
		return self.primeTicks

class CircleCycleForOnePrime:
	primeNumber = 0
	center = (0,0)
#	center = (0.5,0.5)

	logScaling = False
	radius = 0
	currentIndicatorValue = 0
	RING_WIDTH = 2
	INDICATOR_WIDTH = RING_WIDTH * 2
	ANGLE_TO_INDICATE_ZERO = 270.0
	zeroWedge = None
	nonZeroWedge = None
	indicatorWedge = None
	modIndicatorZero=False
	unitSlice = None
	axesForCycle = None

	def __init__(self, center, primeNumber, axes, logScaling):
		self.center = center
		self.primeNumber = int( primeNumber)
		self.setLogScaling(logScaling)
		self.axesForCycle = axes
		self.constructPrimeModCirclePatches()
		self.setIndicator(0)
	
	def setLogScaling(self, useLogScaling, minimumRadius=0):
		self.logScaling = useLogScaling
		self.radius = 2* self.primeNumber
		if (self.logScaling):
			self.radius = math.log(self.primeNumber)
		if (self.radius < minimumRadius):
			self.radius = minimumRadius + 2
		
	def constructPrimeModCirclePatches(self):
		#divide a circle into primeNumber slices
		self.unitSlice = 360.0 / self.primeNumber
		zeroRegionBegin= 0  
		zeroRegionEnd = self.unitSlice

		self.zeroWedge=patchArc(self.center, self.radius, self.radius, theta1=zeroRegionBegin, theta2=zeroRegionEnd, linewidth=CircleCycleForOnePrime.RING_WIDTH, angle = CircleCycleForOnePrime.ANGLE_TO_INDICATE_ZERO - (self.unitSlice/2), color='red', animated=False)
		self.axesForCycle.add_patch(self.zeroWedge)

		self.nonZeroWedge=patchArc(self.center, self.radius, self.radius, theta1=zeroRegionEnd, theta2=zeroRegionBegin,linewidth=CircleCycleForOnePrime.RING_WIDTH,angle = CircleCycleForOnePrime.ANGLE_TO_INDICATE_ZERO - (self.unitSlice/2), color='green', animated=False)
		self.axesForCycle.add_patch(self.nonZeroWedge)
		
		self.indicatorWedge = patchArc(self.center, self.radius, self.radius, theta1= -self.unitSlice / 5, theta2= self.unitSlice/5, linewidth=CircleCycleForOnePrime.INDICATOR_WIDTH, angle = CircleCycleForOnePrime.ANGLE_TO_INDICATE_ZERO - (self.unitSlice/2), color='blue', animated=True)
		self.axesForCycle.add_patch(self.indicatorWedge)
		artists_to_draw_manually.append(self.indicatorWedge)

	def setIndicator(self, newIndicatorValue = None):
		if (newIndicatorValue != None) :
			#convert the remainder to a percent
			self.currentIndicatorValue = (newIndicatorValue % self.primeNumber)
			self.modIndicatorZero = (self.currentIndicatorValue == 0)
		translatedFromValueToAngle = CircleCycleForOnePrime.ANGLE_TO_INDICATE_ZERO - (self.unitSlice/2) + self.unitSlice*self.currentIndicatorValue
		self.indicatorWedge.set_angle(translatedFromValueToAngle)
		self.axesForCycle.draw_artist(self.indicatorWedge)


def drawArtistCollection(artistCollection):
	for oneArtist in artistCollection:
		ax.draw_artist(oneArtist)


#The main drawing method
def updateClockForNumber(currentNumber, cycleContainer, artists_to_draw_manually):
	#only checxk for a new cycle if this is an integer
	currentNumberIsIntegerAndNotDivisibleByAnyCycle = (currentNumber.is_integer() )
	setCountNumberDisplayed(currentNumber)
	for oneCycle in cycleContainer:
		oneCycle.setIndicator(currentNumber)
		#check if current number is divisible by cycle prime
		currentNumberIsIntegerAndNotDivisibleByAnyCycle = currentNumberIsIntegerAndNotDivisibleByAnyCycle and not oneCycle.modIndicatorZero
		
	fig.canvas.flush_events()
	#if current number isn't divisible by any cycle, it's a prime
	if (currentNumberIsIntegerAndNotDivisibleByAnyCycle):
		#got a new prime
		
		newCycle = CircleCycleForOnePrime(plotCenter, currentNumber, ax, False)
		cycles.append(newCycle)
	fig.canvas.flush_events()
    # re-render the artist, updating the canvas state, but not the screen
	drawArtistCollection(artists_to_draw_manually)
	fig.canvas.flush_events()
    # copy the image to the GUI state, but screen might not be changed yet
	fig.canvas.blit(fig.bbox)
	fig.canvas.flush_events()
	return currentNumberIsIntegerAndNotDivisibleByAnyCycle
	
def setCountNumberDisplayed(currentNumber):
	countingString = '{:2g}'.format(currentNumber)
	numberDisplay.set_text(countingString)
	ax.draw_artist(numberDisplay)

cycles = []
startingValue=2
plt.show(block=False)
ax.xaxis.set_major_locator(PrimeFixedLocator())
ax.yaxis.set_major_locator(PrimeFixedLocator())


twoCycle = CircleCycleForOnePrime(plotCenter, startingValue, ax, False)
cycles.append(twoCycle)
ax.autoscale_view()
# flush any pending GUI events, re-painting the screen if needed
fig.canvas.flush_events()
backGroundImage = fig.canvas.copy_from_bbox(fig.bbox)

# draw the animated artist, this uses a cached renderer
drawArtistCollection(artists_to_draw_manually)
# show the result to the screen, this pushes the updated RGBA buffer from the
# renderer to the GUI framework so you can see it
fig.canvas.blit(fig.bbox)

#driving update loop
numberOfFrames = FRAMES_PER_NUMBER * MAX_NUMBER_TO_CHECK
for animationFrameIndex in range(numberOfFrames):
	currentNumber = startingValue + animationFrameIndex / FRAMES_PER_NUMBER
	# reset the background back in the canvas state, screen unchanged
	fig.canvas.restore_region(backGroundImage)
	fig.canvas.flush_events()
	newPrimeFound = updateClockForNumber(currentNumber, cycles, artists_to_draw_manually)
	if (newPrimeFound):
		ax.autoscale_view()
		# flush any pending GUI events, re-painting the screen if needed
		fig.canvas.flush_events()
		plt.pause(.1)
		backGroundImage = fig.canvas.copy_from_bbox(fig.bbox)

	# you can put a pause in if you want to slow things down
	plt.pause(.05)

Using matplotlib backend: qtagg


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math 
import functools
import matplotlib.animation as animation
import matplotlib.text as matText

from matplotlib.collections import PatchCollection
from matplotlib.patches import Arc as patchArc

%matplotlib auto
#plt.rcParams["animation.html"] = "jshtml"
FRAMES_PER_NUMBER = 20
MAX_NUMBER_TO_CHECK = 50
plotCenter = (0,0)
artists_to_draw_manually = []


fig = plt.figure()
ax = fig.subplots()

xSpine = ax.spines['bottom']
xSpine.set_position('zero')
global backGroundImage 

class CircleCycleForOnePrime:
	primeNumber = 0

	center = (0,0)
	radius = 0
	currentIndicatorValue = 0
	RING_WIDTH = 2
	INDICATOR_WIDTH = RING_WIDTH * 2
	zeroWedge = None
	nonZeroWedge = None
	indicatorWedge = None
	modIndicatorZero=False
	unitSlice = None
	axesForCycle = None

	def __init__(self, center, primeNumber, axes):
		self.center = center
		self.primeNumber = int( primeNumber)
		self.radius = 2*primeNumber
		self.axesForCycle = axes
		self.currentIndicatorValue = 0
		self.constructPrimeModCirclePatches()
		
	def constructPrimeModCirclePatches(self):
		#divide a circle into primeNumber slices
		self.unitSlice = 360.0 / self.primeNumber
		zeroRegionBegin= 0  
		zeroRegionEnd = self.unitSlice

		self.zeroWedge=patchArc(self.center, self.radius, self.radius, theta1=zeroRegionBegin, theta2=zeroRegionEnd, linewidth=CircleCycleForOnePrime.RING_WIDTH, color='red')
		self.axesForCycle.add_patch(self.zeroWedge)

		self.nonZeroWedge=patchArc(self.center, self.radius, self.radius, theta1=zeroRegionEnd, theta2=zeroRegionBegin,linewidth=CircleCycleForOnePrime.RING_WIDTH, color='green')
		self.axesForCycle.add_patch(self.nonZeroWedge)
		
		self.indicatorWedge = patchArc(self.center, self.radius, self.radius, theta1= -self.unitSlice / 5, theta2= self.unitSlice/5, linewidth=CircleCycleForOnePrime.INDICATOR_WIDTH, color='blue', animated=True)
		self.axesForCycle.add_patch(self.indicatorWedge)
		artists_to_draw_manually.append(self.indicatorWedge)

	def setIndicator(self, newIndicatorValue = None):
		if (newIndicatorValue != None) :
			#convert the remainder to a percent
			self.currentIndicatorValue = (newIndicatorValue % self.primeNumber)
			self.modIndicatorZero = (self.currentIndicatorValue == 0)
		translatedFromValueToAngle = self.unitSlice*self.currentIndicatorValue
		self.indicatorWedge.set_angle(translatedFromValueToAngle)
		self.axesForCycle.draw_artist(self.indicatorWedge)


def drawArtistCollection(artistCollection):
	for oneArtist in artistCollection:
		ax.draw_artist(oneArtist)


cycles = []
startingValue=2
plt.show(block=False)
twoCycle = CircleCycleForOnePrime(plotCenter, startingValue, ax)
cycles.append(twoCycle)
ax.autoscale_view()
# flush any pending GUI events, re-painting the screen if needed
fig.canvas.flush_events()
backGroundImage = fig.canvas.copy_from_bbox(fig.bbox)

# draw the animated artist, this uses a cached renderer
drawArtistCollection(artists_to_draw_manually)
# show the result to the screen, this pushes the updated RGBA buffer from the
# renderer to the GUI framework so you can see it
#fig.canvas.flush_events()
#fig.canvas.blit(fig.bbox)
#driving update loop
numberOfFrames = FRAMES_PER_NUMBER * MAX_NUMBER_TO_CHECK
for animationFrameIndex in range(numberOfFrames):
	currentNumber = startingValue + animationFrameIndex / FRAMES_PER_NUMBER
	# reset the background back in the canvas state, screen unchanged
	fig.canvas.restore_region(backGroundImage)
	#only checxk for a new cycle if this is an integer
	for oneCycle in cycles:
		oneCycle.setIndicator(currentNumber)
	numberToTesetr = currentNumber% 1
	newCycleMade = numberToTesetr < .001	
	if (newCycleMade ):
		newCycle = CircleCycleForOnePrime(plotCenter, currentNumber, ax)
		cycles.append(newCycle)
    # re-render the artist, updating the canvas state, but not the screen
	drawArtistCollection(artists_to_draw_manually)
	fig.canvas.flush_events()
	
    # copy the image to the GUI state, but screen might not be changed yet
	fig.canvas.blit(fig.bbox)
	if (newCycleMade):
		ax.autoscale_view()
		# flush any pending GUI events, re-painting the screen if needed
#		fig.canvas.flush_events()
#		backGroundImage = fig.canvas.copy_from_bbox(fig.bbox)
	backGroundImage = fig.canvas.copy_from_bbox(fig.bbox)
	print("bbox size " , fig.bbox)
	# you can put a pause in if you want to slow things down
	plt.pause(.05)